# Step 3: Analyze Collected Data

This notebook analyzes the CSV data collected from the naive approaches.

Goals:
1. Load stride_data.csv and naive_trajectory.csv
2. Visualize the trajectory
3. Analyze sensor patterns
4. Identify issues with naive approach
5. Prepare for Bayesian implementation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load Trajectory Data

In [ ]:
# Load naive trajectory
trajectory = pd.read_csv('naive_trajectory.csv')
trajectory.head()

## 2. Plot Trajectory (Top-down view)

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(trajectory['x'], trajectory['y'], 'b-o', linewidth=2, markersize=8, label='Naive DR Path')
plt.plot(0, 0, 'go', markersize=15, label='Start')
plt.plot(trajectory['x'].iloc[-1], trajectory['y'].iloc[-1], 'ro', markersize=15, label='End')

# Add stride numbers
for idx, row in trajectory.iterrows():
    if idx % 2 == 0:  # Label every other stride to avoid clutter
        plt.annotate(f"{int(row['stride'])}", 
                    (row['x'], row['y']), 
                    textcoords="offset points", 
                    xytext=(5,5), 
                    fontsize=8)

plt.xlabel('X Position (m)', fontsize=12)
plt.ylabel('Y Position (m)', fontsize=12)
plt.title('Naive Dead Reckoning Trajectory', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.tight_layout()
plt.show()

## 3. Analyze Heading Changes

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Heading over strides
ax1.plot(trajectory['stride'], trajectory['heading_deg'], 'b-o', linewidth=2)
ax1.set_xlabel('Stride Number', fontsize=12)
ax1.set_ylabel('Heading (degrees)', fontsize=12)
ax1.set_title('Heading Angle Over Time', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Heading change between strides
heading_change = trajectory['heading_deg'].diff()
ax2.plot(trajectory['stride'][1:], heading_change[1:], 'r-o', linewidth=2)
ax2.axhline(y=0, color='k', linestyle='--', alpha=0.3)
ax2.set_xlabel('Stride Number', fontsize=12)
ax2.set_ylabel('Heading Change (degrees)', fontsize=12)
ax2.set_title('Heading Change Between Strides', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Average heading change: {heading_change.abs().mean():.2f}°")
print(f"Max heading change: {heading_change.abs().max():.2f}°")

## 4. Load and Analyze Sensor Data

In [ ]:
# Load stride sensor data
sensor_data = pd.read_csv('stride_data.csv')
sensor_data.head()

In [ ]:
# Plot accelerometer and gyroscope data
fig, axes = plt.subplots(3, 2, figsize=(14, 10))

# Accelerometer
axes[0, 0].plot(sensor_data['stride_number'], sensor_data['accel_x'], 'r-o', label='X')
axes[0, 0].plot(sensor_data['stride_number'], sensor_data['accel_y'], 'g-o', label='Y')
axes[0, 0].plot(sensor_data['stride_number'], sensor_data['accel_z'], 'b-o', label='Z')
axes[0, 0].set_title('Accelerometer (g)', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Gyroscope
axes[0, 1].plot(sensor_data['stride_number'], sensor_data['gyro_x'], 'r-o', label='X')
axes[0, 1].plot(sensor_data['stride_number'], sensor_data['gyro_y'], 'g-o', label='Y')
axes[0, 1].plot(sensor_data['stride_number'], sensor_data['gyro_z'], 'b-o', label='Z')
axes[0, 1].set_title('Gyroscope (deg/s)', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Magnetometer
axes[1, 0].plot(sensor_data['stride_number'], sensor_data['mag_x'], 'r-o', label='X')
axes[1, 0].plot(sensor_data['stride_number'], sensor_data['mag_y'], 'g-o', label='Y')
axes[1, 0].plot(sensor_data['stride_number'], sensor_data['mag_z'], 'b-o', label='Z')
axes[1, 0].set_title('Magnetometer (µT)', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Orientation
axes[1, 1].plot(sensor_data['stride_number'], sensor_data['pitch_deg'], 'r-o', label='Pitch')
axes[1, 1].plot(sensor_data['stride_number'], sensor_data['roll_deg'], 'g-o', label='Roll')
axes[1, 1].plot(sensor_data['stride_number'], sensor_data['yaw_deg'], 'b-o', label='Yaw')
axes[1, 1].set_title('Orientation (degrees)', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Acceleration magnitude
accel_mag = np.sqrt(sensor_data['accel_x']**2 + sensor_data['accel_y']**2 + sensor_data['accel_z']**2)
axes[2, 0].plot(sensor_data['stride_number'], accel_mag, 'purple', linewidth=2, marker='o')
axes[2, 0].axhline(y=1.0, color='k', linestyle='--', label='1g reference')
axes[2, 0].set_title('Acceleration Magnitude', fontweight='bold')
axes[2, 0].set_ylabel('|a| (g)')
axes[2, 0].legend()
axes[2, 0].grid(True, alpha=0.3)

# Gyro magnitude
gyro_mag = np.sqrt(sensor_data['gyro_x']**2 + sensor_data['gyro_y']**2 + sensor_data['gyro_z']**2)
axes[2, 1].plot(sensor_data['stride_number'], gyro_mag, 'orange', linewidth=2, marker='o')
axes[2, 1].set_title('Gyroscope Magnitude', fontweight='bold')
axes[2, 1].set_ylabel('|ω| (deg/s)')
axes[2, 1].grid(True, alpha=0.3)

for ax in axes.flat:
    ax.set_xlabel('Stride Number')

plt.tight_layout()
plt.show()

## 5. Problems with Naive Approach

List the issues you observe:

1. **Heading drift**: Does the yaw angle drift over time?
2. **Position error accumulation**: How much does the final position differ from reality?
3. **No map constraints**: The path might go through walls
4. **Fixed stride length**: Not all steps are equal

Write your observations here:

### Your Observations:

- 
- 
- 

## 6. Next Steps

Based on your observations, you'll implement:

1. **Kalman Filter** - Filter noisy sensor readings
2. **Bayesian Filter** - Use floor plan to constrain position estimates
3. **Particle Filter** - Compare with Bayesian approach

This will significantly improve accuracy!